In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import pandas as pd
import datetime as dt
from pymongo import MongoClient
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.functions import col, split, element_at, when
import pygeohash as pgh
import json

In [2]:
spark = (
    SparkSession.builder
    .master('local[*]')    #import all local processor
    .appName('Spark Streaming from Kafka into MongoDB')
    .getOrCreate() 
)

In [3]:
topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')   #same environment from producer
    .option('subscribe', "Asgn_PartB")    #topic = Asgn_PartB
    .load()
)

In [4]:
topic_stream_df.printSchema() 
#we will focus on the key and value
#the actual reading data will be from "value"

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
StringDF = topic_stream_df.selectExpr("CAST(key AS STRING)", 
                                      "CAST(value AS STRING)")

In [6]:
# References: https://gankrin.org/how-to-read-kafka-json-data-in-spark-structured-streaming/

from pyspark.sql.types import *
from pyspark.sql.functions import from_json
#Cast JSON to stringType  
schema = StructType()       \
        .add("latitude",DoubleType()) \
        .add("longitude",DoubleType()) \
        .add("air_temperature_celcius", FloatType()) \
        .add("relative_humidity", FloatType()) \
        .add("windspeed_knots", FloatType()) \
        .add('max_wind_speed', FloatType()) \
        .add('precipitation', FloatType()) \
        .add('precipitation_type', StringType()) \
        .add('GHI_w', FloatType())\
        .add("confidence",FloatType()) \
        .add("surface_temperature_celcius",FloatType()) \
        .add("created_date",TimestampType()) 

#alias == rename
DF = StringDF.select(
    col("key").alias("key"),
    from_json(col("value"), schema).alias("data")
)

In [7]:
def removeNullvalue(d):
    '''
    d: dictionary format
    To remove the None value from the dictionary
    '''
    return {k:v for k, v in d.items() if v}
        
def process_data(batch_df, batch_id):
    '''
    To process the data accordings to the requirement
    '''
    raw_data = batch_df.collect()
    #raw data = [Row(), Row()]
    #process data
    #data = [{}, {}]
    climate = {}
    AQUA_lst = []
    TERRA_lst = []
    #Raw_data will start with empty list, hence we ignore it
    try:
        if len(raw_data) > 0:
            for raw in raw_data:
                key = raw['key']
                value = raw['data']
                value_dic = raw["data"].asDict()
                clean_dic = removeNullvalue(value_dic)
                #print(clean_dic)
                if key == 'producer1':
                    climate = clean_dic
                elif key == 'producer2':
                    AQUA_lst.append(clean_dic)
                else:
                    TERRA_lst.append(clean_dic)

                #print("Climate data: ", climate)
                #print("AQUA hotspot data: ", AQUA_lst)
                #print("TERRA hotspot data: ", TERRA_lst)

                #assigment logic
                #1) check if climate data in data array
                #if not, return

                #2) climate = {}, hotspot = [{}, {}]
                #compute geohash and compare
                #print("Checking")
                hotspot = []
                if climate != {}:
                    cli_longtitude = climate['longitude']
                    cli_latitude = climate['latitude']
                    encode_cli = pgh.encode(latitude=cli_latitude, longitude=cli_longtitude, precision=3)
                    for record in AQUA_lst:
                        encode_AQUA = pgh.encode(latitude=record['latitude'], longitude=record['longitude'], precision=3)
                        if encode_AQUA == encode_cli:
                            record['satellites'] = 'AQUA'
                            hotspot.append(record)
                    for record in TERRA_lst:
                        encode_TERRA = pgh.encode(latitude=record['latitude'], longitude=record['longitude'], precision=3)
                        if encode_TERRA == encode_cli:
                            record['satellites'] = 'TERRA'
                            hotspot.append(record)
                            #print("Record", record)
                    #print(hotspot)

                #4) Group hotspot data by location and aggregate of surface temperature and confidence
                    if len(hotspot) >=2:
                        print("there is fire?")
                        grp_hotspot = []
                        
                        AQUA_sate = []
                        TERRA_sate = []
                        for data in hotspot:
                            if data['satellites'] == 'AQUA':
                                AQUA_sate.append(data)
                            elif data['satellites'] == 'TERRA':
                                TERRA_sate.append(data)
                        
                        if len(AQUA_sate)>0 and len(TERRA_sate)>0:
                            for AQUA in AQUA_sate:
                                aqua_latitude = AQUA['latitude']
                                aqua_longitude = AQUA['longitude']
                                for TERRA in TERRA_sate:
                                    encode_AQUA5 = pgh.encode(latitude=aqua_latitude, longitude=aqua_longitude, precision=5)
                                    encode_TERRA5 = pgh.encode(latitude=TERRA['latitude'], longitude=TERRA['longitude'], precision=5)
                                    if encode_AQUA5 == encode_TERRA5:
                                        fire = {}
                                        fire['avg_temperature'] = (AQUA['surface_temperature_celcius'] + TERRA['surface_temperature_celcius'])/2
                                        fire['avg_confidence'] = (AQUA['confidence'] + TERRA['confidence'])/2
                                        fire['latitude'] = aqua_latitude
                                        fire['longitude'] = aqua_longitude

                                        grp_hotspot.append(fire)
                                        print("There is a fire event here!", grp_hotspot)
                                        hotspot = grp_hotspot
                                        
                    #5) Detect Cause of Fire
                    if len(hotspot) >0:
                        air_temp = climate['air_temperature_celcius']
                        GHI = climate['GHI_w']
                        event = 'others'
                        if air_temp > 20 and GHI > 180: 
                            event  = 'natural'

                        for data in hotspot:
                            print(data)
                            data['event'] = event
                                        
                    climate['hotspot'] = hotspot

                    #Climate = {}, hotspot = [{}] or [{}, {}, {}]
                    #6) Store Climate and Hotspots in MongoDB            
                    try:
                        db.hotspot.insert_one(climate)
                    except Exception as ex:
                        print(ex)
    except Exception as ex:
        print(ex)

In [8]:
db_writer = (
    DF
    .writeStream
    .outputMode('append')
    .trigger(processingTime='10 seconds')   #batch in 10 seconds
    .foreachBatch(process_data)             #for each batch, it will process the data
)

In [9]:
mongo_client = MongoClient()

db = mongo_client.DataImport
db.hotspot.drop()

In [10]:
Streaming = db_writer.start()

In [11]:
import time
Streaming = db_writer.start()
time.sleep(500)
Streaming.stop()

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7ae36f426f9719ddcdf') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f7ae36f426f9719ddcdf')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7ae36f426f9719ddcdf') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7ae36f426f9719ddcdf') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f7ae36f426f9719ddcdf')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7ae36f426f9719ddcdf') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7ae36f426f9719ddce0') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

{'latitude': -37.457, 'longitude': 148.143, 'confidence': 90.0, 'surface_temperature_celcius': 41.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 31, 47, 634193), 'satellites': 'TERRA'}
{'latitude': -37.457, 'longitude': 148.143, 'confidence': 90.0, 'surface_temperature_celcius': 41.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 31, 47, 634193), 'satellites': 'TERRA', 'event': 'others'}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7d636f426f9719ddce7') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f7d636f426f9719ddce7')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f7d636f426f9719ddce7') }"}
{'latitude': -37.457, 'longitude': 148.143, 'confidence': 90.0, 'surface_temperature_celcius': 41.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 31, 47, 634193), 'satellites': 'TERRA', 'event': 'others'}


E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f81c36f426f9719ddcef') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f81c36f426f9719ddcef')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f81c36f426f9719ddcef') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f81c36f426f9719ddcef') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f81c36f426f9719ddcef')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f81c36f426f9719ddcef') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f81c36f426f9719ddcef') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f83036f426f9719ddcf3') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f83036f426f9719ddcf3')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f83036f426f9719ddcf3') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f83036f426f9719ddcf3') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f83036f426f9719ddcf3')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f83036f426f9719ddcf3') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f83036f426f9719ddcf3') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f84436f426f9719ddcf7') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f84436f426f9719ddcf7')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f84436f426f9719ddcf7') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f84436f426f9719ddcf8') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f84436f426f9719ddcf8')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f84436f426f9719ddcf8') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f84436f426f9719ddcf7') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

...

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f96636f426f9719ddd31') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f96636f426f9719ddd31')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f96636f426f9719ddd31') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f96636f426f9719ddd32') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f96636f426f9719ddd32')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f96636f426f9719ddd32') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f96636f426f9719ddd31') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f97a36f426f9719ddd35') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f97a36f426f9719ddd35')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f97a36f426f9719ddd35') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f97a36f426f9719ddd35') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f97a36f426f9719ddd35')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f97a36f426f9719ddd35') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f97a36f426f9719ddd35') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

there is fire?
{'latitude': -36.1188, 'longitude': 145.156, 'confidence': 65.0, 'surface_temperature_celcius': 42.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 38, 35, 634193), 'satellites': 'TERRA'}
{'latitude': -36.5141, 'longitude': 145.0292, 'confidence': 62.0, 'surface_temperature_celcius': 41.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 38, 37, 634193), 'satellites': 'TERRA'}
there is fire?
{'latitude': -36.1188, 'longitude': 145.156, 'confidence': 65.0, 'surface_temperature_celcius': 42.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 38, 35, 634193), 'satellites': 'TERRA', 'event': 'natural'}
{'latitude': -36.5141, 'longitude': 145.0292, 'confidence': 62.0, 'surface_temperature_celcius': 41.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 38, 37, 634193), 'satellites': 'TERRA', 'event': 'natural'}
there is fire?
{'latitude': -36.1188, 'longitude': 145.156, 'confidence': 65.0, 'surface_temperature_celcius': 42.0, 'created_date': datetime.datetime(2022, 5, 

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f99836f426f9719ddd3b') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f99836f426f9719ddd3b')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f99836f426f9719ddd3b') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f99836f426f9719ddd3b') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f99836f426f9719ddd3b')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f99836f426f9719ddd3b') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f99836f426f9719ddd3b') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ac36f426f9719ddd3f') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9ac36f426f9719ddd3f')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ac36f426f9719ddd3f') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ac36f426f9719ddd3f') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9ac36f426f9719ddd3f')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ac36f426f9719ddd3f') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ac36f426f9719ddd3f') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

In [12]:
#Streaming.stop()

In [13]:
from pprint import pprint
cursor = db.hotspot.find({})
for document in cursor:
    pprint(document)

{'GHI_w': 83.0,
 '_id': ObjectId('6288f7ae36f426f9719ddcdf'),
 'air_temperature_celcius': 9.0,
 'created_date': datetime.datetime(2019, 1, 4, 0, 0),
 'hotspot': [],
 'latitude': -37.559,
 'longitude': 148.037,
 'max_wind_speed': 19.0,
 'precipitation': 0.23999999463558197,
 'precipitation_type': 'G',
 'relative_humidity': 41.099998474121094,
 'windspeed_knots': 12.699999809265137}
{'GHI_w': 83.0,
 '_id': ObjectId('6288f7ae36f426f9719ddce0'),
 'air_temperature_celcius': 9.0,
 'created_date': datetime.datetime(2019, 1, 4, 0, 0),
 'hotspot': [],
 'latitude': -37.559,
 'longitude': 148.037,
 'max_wind_speed': 19.0,
 'precipitation': 0.23999999463558197,
 'precipitation_type': 'G',
 'relative_humidity': 41.099998474121094,
 'windspeed_knots': 12.699999809265137}
{'GHI_w': 194.0,
 '_id': ObjectId('6288f7b836f426f9719ddce1'),
 'air_temperature_celcius': 22.0,
 'created_date': datetime.datetime(2019, 1, 5, 0, 0),
 'hotspot': [],
 'latitude': -37.062,
 'longitude': 141.373,
 'max_wind_speed': 1

In [14]:
cursor = db.hotspot.find({}, {"hotspot":1})
for document in cursor:
    pprint(document)

{'_id': ObjectId('6288f7ae36f426f9719ddcdf'), 'hotspot': []}
{'_id': ObjectId('6288f7ae36f426f9719ddce0'), 'hotspot': []}
{'_id': ObjectId('6288f7b836f426f9719ddce1'), 'hotspot': []}
{'_id': ObjectId('6288f7b836f426f9719ddce2'), 'hotspot': []}
{'_id': ObjectId('6288f7c236f426f9719ddce3'),
 'hotspot': [{'confidence': 71.0,
              'created_date': datetime.datetime(2022, 5, 22, 0, 31, 28, 992000),
              'event': 'others',
              'latitude': -36.3017,
              'longitude': 145.6213,
              'satellites': 'AQUA',
              'surface_temperature_celcius': 46.0}]}
{'_id': ObjectId('6288f7c236f426f9719ddce4'),
 'hotspot': [{'confidence': 71.0,
              'created_date': datetime.datetime(2022, 5, 22, 0, 31, 28, 992000),
              'event': 'others',
              'latitude': -36.3017,
              'longitude': 145.6213,
              'satellites': 'AQUA',
              'surface_temperature_celcius': 46.0}]}
{'_id': ObjectId('6288f7cc36f426f9719ddce5')

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9b636f426f9719ddd41') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9b636f426f9719ddd41')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9b636f426f9719ddd41') }"}
{'latitude': -36.8679, 'longitude': 142.7328, 'confidence': 59.0, 'surface_temperature_celcius': 40.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 39, 19, 634193), 'satellites': 'TERRA'}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9b636f426f9719ddd41') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9b636f426f9719ddd41')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9b636f426f9719ddd41') }"}
{'latitude': -36.8679, 'longi

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ca36f426f9719ddd43') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9ca36f426f9719ddd43')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ca36f426f9719ddd43') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ca36f426f9719ddd43') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9ca36f426f9719ddd43')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ca36f426f9719ddd43') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9ca36f426f9719ddd43') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

{'latitude': -36.9455, 'longitude': 141.849, 'confidence': 77.0, 'surface_temperature_celcius': 51.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 40, 18, 992075), 'satellites': 'AQUA'}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9f236f426f9719ddd47') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288f9f236f426f9719ddd47')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9f236f426f9719ddd47') }"}
{'latitude': -36.9455, 'longitude': 141.849, 'confidence': 77.0, 'surface_temperature_celcius': 51.0, 'created_date': datetime.datetime(2022, 5, 22, 0, 40, 18, 992075), 'satellites': 'AQUA', 'event': 'others'}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288f9f236f426f9719ddd47') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id'

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa1036f426f9719ddd4a') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288fa1036f426f9719ddd4a')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa1036f426f9719ddd4a') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa1036f426f9719ddd4a') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288fa1036f426f9719ddd4a')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa1036f426f9719ddd4a') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa1036f426f9719ddd4a') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('

E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa2e36f426f9719ddd4d') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288fa2e36f426f9719ddd4d')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa2e36f426f9719ddd4d') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa2e36f426f9719ddd4d') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6288fa2e36f426f9719ddd4d')}, 'errmsg': "E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa2e36f426f9719ddd4d') }"}
E11000 duplicate key error collection: DataImport.hotspot index: _id_ dup key: { _id: ObjectId('6288fa2e36f426f9719ddd4d') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('